In [33]:
#Expand notebook to take full screen width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Fixing Filenames

In [35]:
import subprocess
import glob, os
os.chdir("./models/")

In [36]:
file_list = []
for file in glob.glob("*mnist*"):
    file_list.append(file)

In [37]:
for filename in file_list:
    temp_str = filename.split("_")[7]
    mix_str = filename.split("_")[8]
    #print (filename, temp_str, mix_str)
    if 'm' in temp_str:
        temp = "kdT" + str(int(float(temp_str.split("m")[1])))
        mix = "m" + mix_str.split("kdT")[1]
        new_filename = filename.replace(mix_str, temp).replace(temp_str, mix)
        subprocess.call("mv {} {}".format(filename, new_filename).split(" "))

### Removing temp from sobol sequence and deduping

In [2]:
import numpy as np
import pickle

with open("../sobol_search.p", "rb") as handle:
    params = pickle.load(handle)

tupled_params = [tuple(row) for row in np.vstack((params['mean'], params['var'], params['tau'], params['mixtures'])).T]
unique_params = list(set(tupled_params))

reduced_params = {}
reduced_params['mean'] = np.array([x[0] for x in unique_params])
reduced_params['var'] = np.array([x[1] for x in unique_params])
reduced_params['tau'] = np.array([x[2] for x in unique_params])
reduced_params['mixtures'] = np.array([int(x[3]) for x in unique_params])

### Output to CSV

In [31]:
import sys
sys.path.insert(0,'../../src/')
import os
import argparse
from retrain_model import retrain_model
savedir = os.getcwd() + "/models/"

import pickle
from mnist_loader import train_data
from utils_sws import sws_prune, compressed_model
from utils_model import test_accuracy
import torch
from torch.autograd import Variable
from mnist_loader import search_train_data, search_retrain_data, search_validation_data, train_data, test_data, batch_size

from time import time

In [34]:
test_data_full =  Variable(test_data(fetch = "data")).cuda()
test_labels_full =  Variable(test_data(fetch = "labels")).cuda()
val_data_full =  Variable(search_validation_data(fetch = "data")).cuda()
val_labels_full =  Variable(search_validation_data(fetch = "labels")).cuda()
start = 0
end = 10

with open("../sobol_search.p", "rb") as handle:
    params = pickle.load(handle)
for i in range (start,end):
    print ("mean: {}, var: {}, tau: {}, temp: {}, mixtures: {}".format(params['mean'][i], params['var'][i], params['tau'][i], float(params['temp'][i]), int(params['mixtures'][i])))
    mean = float(params['mean'][i])
    var = float(params['var'][i])
    beta = mean/var
    alpha = mean * beta

    exp_name = "{}_a{}_b{}_r{}_t{}_m{}_kdT{}_{}".format('SWSModel', alpha, beta, 50, float(params['tau'][i]), int(params['mixtures'][i]), int(params['temp'][i]), 'search')
    model_file = "./models/mnist_retrain_{}".format(exp_name)
    if not os.path.exists("{}.m".format(model_file)):
        print ("File not found: {}.m".format(model_file))
    else:
        model = torch.load("{}.m".format(model_file)).cuda()
        with open("{}_gmp.p".format(model_file), "rb") as handle:
            gmp = pickle.load(handle)
        model.load_state_dict(sws_prune(model, gmp))
        test_acc = test_accuracy(test_data_full, test_labels_full, model)[0]
        val_acc = test_accuracy(val_data_full, val_labels_full, model)[0]
        cm = compressed_model(model.state_dict(), [gmp])
        cr = cm.get_cr(6)[0]
        sp = (cm.binned_weights == 0).sum() / float(cm.binned_weights.size) * 100.0
        if not os.path.exists("results.csv"):
            with open("results.csv", "w") as out_csv:
                out_csv.write("Exp, Mean, Var, Tau, Temp, Mixtures, Test Acc, Val Acc, Sparse, CR\n")
                out_csv.write(", ".join([str(x) for x in [i, params['mean'][i], params['var'][i], params['tau'][i], int(params['temp'][i]), int(params['mixtures'][i]), test_acc, val_acc, sp, cr]]) + "\n")
        else:
            with open("results.csv", "a") as out_csv:
                out_csv.write(", ".join([str(x) for x in [i, params['mean'][i], params['var'][i], params['tau'][i], int(params['temp'][i]), int(params['mixtures'][i]), test_acc, val_acc, sp, cr]]) + "\n")

mean: 0.05725487884358381, var: 0.3043676123956402, tau: 3.9153756672351225e-06, temp: 14.0, mixtures: 6
./models/mnist_retrain_SWSModel_a0.010770269298995945_b0.18811094384496982_r50_t3.9153756672351225e-06_m6_kdT14_search
mean: 18105.58243027124, var: 0.3043676123956402, tau: 3.9153756672351225e-06, temp: 14.0, mixtures: 6
./models/mnist_retrain_SWSModel_a1077026929.899596_b59485.903535413694_r50_t3.9153756672351225e-06_m6_kdT14_search
mean: 0.05725487884358381, var: 0.16962590880724215, tau: 3.9153756672351225e-06, temp: 14.0, mixtures: 6
./models/mnist_retrain_SWSModel_a0.01932559226620634_b0.33753616559040256_r50_t3.9153756672351225e-06_m6_kdT14_search
mean: 0.05725487884358381, var: 0.3043676123956402, tau: 0.0003141017185010456, temp: 14.0, mixtures: 6
./models/mnist_retrain_SWSModel_a0.010770269298995945_b0.18811094384496982_r50_t0.0003141017185010456_m6_kdT14_search
mean: 0.05725487884358381, var: 0.3043676123956402, tau: 3.9153756672351225e-06, temp: 11.0, mixtures: 6
./model